In [5]:
import pandas as pd
import pathlib
import numpy as np

In [8]:
caminho_bases = pathlib.Path('archive')
base_airbnb = pd.DataFrame()
meses = {'jan' : 1, 'fev' : 2, 'mar' : 3, 'abr' : 4, 'mai' : 5, 'jun' : 6, 'jul' : 7, 'ago' : 8, 'set' : 9, 'out': 10, 'nov' : 11, 'dez' : 12}


for arquivo in caminho_bases.iterdir():
    nome_mes = arquivo.name[:3]
    mes = meses[nome_mes]
    ano = arquivo.name[-8:]
    ano = int(ano.replace('.csv', ''))
    
    df = pd.read_csv(caminho_bases / arquivo.name)
    
    df['ano'] = ano
    df['mes'] = mes

    base_airbnb = base_airbnb._append(df)



/var/folders/qz/y6_hsdhd6gsbs7nck_9v8f8r0000gn/T/ipykernel_1986/4254408390.py:12: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_bases / arquivo.name)
/var/folders/qz/y6_hsdhd6gsbs7nck_9v8f8r0000gn/T/ipykernel_1986/4254408390.py:12: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_bases / arquivo.name)
/var/folders/qz/y6_hsdhd6gsbs7nck_9v8f8r0000gn/T/ipykernel_1986/4254408390.py:12: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_bases / arquivo.name)


KeyError: '.DS'

- Temos muitas colunas, então vou analisar para usar apenas as colunas necessárias para a previsão dos preços.

- Para isso, vou criar um arquivo em excel com os 1000 primeiros registros e fazer uma analise qualitativa.

- Tipos de informações que eu vou excluir:
-IDS, Links, Informações não relevantes
-Colunas repetidadas ou extremamente parecidas com outras (que dão a mesma informação para o modelo)
-Colunas preenchidas com texto livre (não irá ser rodado nenhuma análise de texto)
-Coluna em que todos ou quase todos os valores são iguais


In [ ]:
colunas = ['host_response_time','host_response_rate','host_is_superhost','host_listings_count','latitude','longitude','property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type','amenities','price','security_deposit','cleaning_fee','guests_included','extra_people','minimum_nights','maximum_nights','number_of_reviews','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','instant_bookable','is_business_travel_ready','cancellation_policy','ano','mes']

base_airbnb = base_airbnb.loc[:, colunas]
display(base_airbnb)

Verificando a quantidade de valores nulos na minha base da dados e fazendo o tratamento 

In [ ]:

for coluna in base_airbnb:
    if base_airbnb[coluna].isnull().sum() > 300000:
        base_airbnb = base_airbnb.drop(coluna, axis=1)
        


In [ ]:
base_airbnb = base_airbnb.dropna()
print(base_airbnb.isnull().sum())

Como preço e extra people estão sendo reconhecidos como objeto, tenho que mudar o tipo de variável da coluna

In [ ]:
print(base_airbnb.dtypes)
print('-'*60)
print(base_airbnb.iloc[0])


In [ ]:
# Certificar-se de que as colunas são strings
base_airbnb['price'] = base_airbnb['price'].astype(str)
base_airbnb['extra_people'] = base_airbnb['extra_people'].astype(str)


# Remover símbolos de dólar e vírgulas, em seguida converter para float
base_airbnb['price'] = base_airbnb['price'].str.replace('$', '').str.replace(',', '').astype(np.float32, copy=False)
base_airbnb['extra_people'] = base_airbnb['extra_people'].str.replace('$', '').str.replace(',', '').astype(np.float32, copy=False)

base_airbnb['price'] = pd.to_numeric(base_airbnb['price'], errors='coerce')
base_airbnb['extra_people'] = pd.to_numeric(base_airbnb['extra_people'], errors='coerce')
print(base_airbnb.dtypes)

In [ ]:
print(base_airbnb.corr())